## Test - EPA HUB

In [37]:
import requests
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup

### Parameters

In [4]:
URL = "https://www.epa.gov/climateleadership/ghg-emission-factors-hub"

### Functions

In [6]:
headers = {
    "Authority": "www.epa.gov",
    "Method": "GET",
    "Path": "/climateleadership/ghg-emission-factors-hub",
    "Scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "If-None-Match": "1689275341",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}


### Tests

In [45]:
r = requests.get(URL, headers=headers)
soup = BeautifulSoup(r.content, "html.parser")
# Get all hrefs and their text
href_dicts = []
hrefs = soup.find_all("a")
for href in hrefs:
    href_dict = {}
    text=  href.text
    url = href.get("href")

    if "GHG" not in text:
        continue

    if "xls" in url:     
        href_dict["name"] = str(text).replace("\n", "")
        href_dict["url"] =  f"https://www.epa.gov{url}"
        href_dicts.append(href_dict)
 
    
    elif "pdf" in url:
        href_dict["name"] = str(text).replace("\n", "") 
        href_dict["url"] = f"https://www.epa.gov{url}"

        href_dicts.append(href_dict)

# Create dataframe
df = pd.DataFrame.from_dict(href_dicts)

In [46]:
df

,name,url
0,2023 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
1,2023 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
2,ARCHIVED 2022 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
3,ARCHIVED 2022 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
4,ARCHIVED 2021 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
5,ARCHIVED 2021 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
6,ARCHIVED 2020 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
7,ARCHIVED 2020 GHG Emission Factors Hub (pdf),https://www.epa.gov/sites/default/files/2021-0...
8,ARCHIVED 2018 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
9,ARCHIVED 2018 GHG Emission Factors Hub (pdf),https://www.epa.gov/sites/default/files/2018-0...


### Getting xlsx files

In [47]:
xlsx_urls = df["url"].tolist()
undesired_words = ["pdf"]

xlsx_urls = [url for url in xlsx_urls if not any(word in url for word in undesired_words)]
xlsx_urls

['https://www.epa.gov/system/files/documents/2023-03/ghg-emission-factors-hub.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2022.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2021.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2020.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2018.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2015.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2014.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2011.xlsx']

In [99]:
mega_df = pd.DataFrame([])
for xlsx_url in xlsx_urls[:1]:
    response = requests.get(xlsx_url, headers=headers)
    xls_file = pd.ExcelFile(BytesIO(response.content))
    # Create a dictionary of DataFrames, with sheet name as key
    dataframes = {sheet_name: xls_file.parse(sheet_name) for sheet_name in xls_file.sheet_names}
    key_of_interest = None
    for key in dataframes.keys():
        if "Hub" in key:
            key_of_interest = key

    if key_of_interest is None:
        print(f"Skipping {xlsx_url}")
        continue

    ghg_data = dataframes[key_of_interest]
    ghg_data = ghg_data.dropna(axis=1, how="all")
    # Remove rows with several NaNs
    ghg_data = ghg_data.dropna(axis=0, thresh=2)

c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [100]:
ghg_data

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
5,NaN,NaN,Gas,100-Year GWP,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,CH4,25,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,N2O,298,NaN,NaN,NaN,NaN,NaN
10,Table 1,Stationary Combustion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,Fuel Type,Heat Content (HHV),CO2 Factor,CH4 Factor,N2O Factor,CO2 Factor,CH4 Factor,N2O Factor
...,...,...,...,...,...,...,...,...,...
571,NaN,R-502,0,"48.8% HCFC-22 , 51.2% CFC-115",NaN,NaN,NaN,NaN,NaN
572,NaN,R-504,325,"48.2% HFC-32 , 51.8% CFC-115",NaN,NaN,NaN,NaN,NaN
573,NaN,R-507,3985,"5% HFC-125 , 5% HFC143a",NaN,NaN,NaN,NaN,NaN
574,NaN,R-508A,13214,"39% HFC-23 , 61% PFC-116",NaN,NaN,NaN,NaN,NaN
